In [1]:
import os
os.chdir(f"../../..")
from SOURCE.CLASS_CODE_GPU_Classes import *
from SOURCE.CLASS_CODE_Image_Manager import *
import numpy as np
import json
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from IPython import display
from time import time
import h5py

In [2]:
# Set the PARAMETERS ############################################
##################################################################
experiment_name="Basler_like_R0_300x_w0_300x_Z_50x_64bit"
N_R0 = 5
N_w0 = 5
N_Z = 2

output_directory=f"./OUTPUT/LIBRARIES_OF_THEORETICAL_D/{experiment_name}/"
#os.chdir(f"/home/oiangu/Desktop/Conical_Refraction_Polarimeter/")

randomization_seed=666
image_shortest_side=int(540/3)
saturation=1

# Ring parameters to test (each will be a different simulation)
#phiCR_s=np.linspace(-180,180,360*10**significant_decimal+1)*np.pi/180
R0_s=np.linspace(70, 180, N_R0)/3 #np.linspace(70,180,40) # in pxels 153
w0_s=np.linspace( 8, 50, N_w0)/3 #np.linspace(8,50,40) 11
Z_s=np.linspace(0, 1, N_Z)/3
rho_0s=R0_s/w0_s


resolution_side_nx=image_shortest_side # generated images will be resolution_side x resolution_side
# Other parameters
max_k=50
num_k=1200
sim_chunk_ax=image_shortest_side



##################################################
##################################################

# General considerations
image_directory=f"{output_directory}/SIMULATIONS/" #nx_{image_shortest_side}_depth_{image_depth}_sat_{saturation}
os.makedirs(image_directory, exist_ok=True)
np.random.seed(randomization_seed)

In [3]:
# Initialize the vigilant
try:
    phase_vigilant = json.load(open(f"{output_directory}/STRUCTURE_Grid.json"))
except:
    phase_vigilant = {'R0s':[], 'w0s':[], 'Zs':[], 'IDs':[]}

# Set the objects ready ##################
# The simulator object
simulator=RingSimulator_Optimizer_GPU( n=1.5, a0=1.0, max_k=max_k, num_k=num_k, nx=resolution_side_nx, sim_chunk_x=sim_chunk_ax, sim_chunk_y=sim_chunk_ax)

# Initialize the hdf5 dataset saver
h5f = h5py.File(f"{image_directory}.h5", 'w')

# save phis
h5f.create_dataset('phis', data=simulator.phis[:,:,0], compression="lzf", shuffle=True) #, compression_opts=9)


# Execute the stuff #####################
i=1
total=Z_s.shape[0]*R0_s.shape[0]*w0_s.shape[0]
elapsed=0
beg=time()
output_info_every=100
dump_every=10000

for Z in Z_s:
    for R0 in R0_s:
        for w0 in w0_s:
            ID=f"R0_{R0}_w0_{w0}_Z_{Z}"
            if ID not in phase_vigilant['IDs']:
                # simulate matrix
                D_matrix = simulator.compute_pieces_for_I_LP(R0_pixels=R0, Z=Z, w0_pixels=w0)
                
                if D_matrix is None:
                    raise ValueError
                    
                # save the matrix
                h5f.create_dataset(ID, data=D_matrix, compression="lzf", shuffle=True) #, compression_opts=9)

                #append the data
                phase_vigilant['IDs'].append(ID)
                phase_vigilant['R0s'].append(float(R0))
                phase_vigilant['Zs'].append(float(Z))
                phase_vigilant['w0s'].append(float(w0))
                
                
                if i%output_info_every==0:
                    display.clear_output(wait=True)
                    elapsed=time()-beg
                    print(f"["+'#'*(int(100*i/total))+' '*(100-int(100*i/total))+f"] {100*i/total:3.4}% \n\nSimulated: {i}/{total}\nElapsed time: {elapsed//3600} h {elapsed//60-(elapsed//3600)*60} min {elapsed-(elapsed//60)*60-(elapsed//3600)*60:2.4} s")
                    if i%dump_every==0:
                        # we save the progess (in order to be able to quit and resume)
                        json.dump(phase_vigilant, open( f"{output_directory}/STRUCTURE_Grid.json", "w"))
            i+=1
h5f.close()
print("\n\nFINISHED!!!")

2021-12-09 19:19:02.509948: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:113] *** WARNING *** You are using ptxas 9.1.108, which is older than 9.2.88. ptxas 9.x before 9.2.88 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You do not need to update to CUDA 9.2.88; cherry-picking the ptxas binary is sufficient.
2021-12-09 19:19:02.533429: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:113] *** WARNING *** You are using ptxas 9.1.108, which is older than 9.2.88. ptxas 9.x before 9.2.88 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You do not need to update to CUDA 9.2.88; cherry-picking the ptxas binary is sufficient.
2021-12-09 19:19:02.555352: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:113] *** WARNING *** You are using ptxas 9.1.108, which is older than 9.2.88. ptxas 9.x before 9.2.88 is known to miscompile XLA code, 

2021-12-09 19:19:05.074909: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:113] *** WARNING *** You are using ptxas 9.1.108, which is older than 9.2.88. ptxas 9.x before 9.2.88 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You do not need to update to CUDA 9.2.88; cherry-picking the ptxas binary is sufficient.
2021-12-09 19:19:05.159026: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:113] *** WARNING *** You are using ptxas 9.1.108, which is older than 9.2.88. ptxas 9.x before 9.2.88 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You do not need to update to CUDA 9.2.88; cherry-picking the ptxas binary is sufficient.
2021-12-09 19:19:05.172580: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:113] *** WARNING *** You are using ptxas 9.1.108, which is older than 9.2.88. ptxas 9.x before 9.2.88 is known to miscompile XLA code, 



FINISHED!!!


In [4]:
ID

'R0_60.0_w0_16.666666666666668_Z_0.3333333333333333'

In [18]:
t=time()
h5f = h5py.File(f"{image_directory}.h5",'r')
D_mat = h5f[ID][:]
phis = h5f['phis'][:]
#print(list(h5f.keys()))
h5f.close()
print(f"hf5:{time()-t}s size {os.path.getsize(image_directory+'.h5')}")   
print(phis.dtype, phis.shape)

<HDF5 file ".h5" (mode r)>
hf5:0.0029730796813964844s size 10295460
<Closed HDF5 file>


In [6]:
905693/4

226423.25

In [8]:
os.path.getsize(image_directory+'.h5')/total

205909.2

In [7]:
import pickle, gzip, lzma, bz2
import h5py


# Initialize the vigilant
try:
    phase_vigilant = json.load(open(f"{output_directory}/STRUCTURE_Grid.json"))
except:
    phase_vigilant = {'R0s':[], 'w0s':[], 'Zs':[], 'IDs':[], 'rel_path':[]}

# Set the objects ready ##################
# The simulator object
simulator=RingSimulator_Optimizer_GPU( n=1.5, a0=1.0, max_k=max_k, num_k=num_k, nx=resolution_side_nx, sim_chunk_x=sim_chunk_ax, sim_chunk_y=sim_chunk_ax)

# Execute the stuff #####################
i=1
total=Z_s.shape[0]*R0_s.shape[0]*w0_s.shape[0]
elapsed=0
beg=time()
output_info_every=100
dump_every=10000

for Z in Z_s:
    for R0 in R0_s:
        for w0 in w0_s:
            ID=f"R0_{R0}_w0_{w0}_Z_{Z}"
            if ID not in phase_vigilant['IDs']:
                # simulate matrix
                #D_matrix = simulator.compute_D_matrix( R0_pixels=R0, Z=Z, w0_pixels=w0)
                D_matrix = simulator.compute_pieces_for_I_LP(R0_pixels=R0, Z=Z, w0_pixels=w0)
                
                # save the matrix
                rel_path=f"{image_directory}/{ID}.pkl.lzma"
                #np.save( rel_path, D_matrix, allow_pickle=False)
                print(f"\nDUMP {i}")
                t=time()
                pickle.dump(D_matrix, lzma.open(rel_path, 'wb'))
                print(f"lzma:{time()-t}s size {os.path.getsize(rel_path)}")
                t=time()
                pickle.dump(D_matrix, gzip.open(rel_path+".gzip", 'wb'))
                print(f"gzip:{time()-t}s size {os.path.getsize(rel_path+'.gzip')}")
                t=time()
                pickle.dump(D_matrix, bz2.open(rel_path+".bz2", 'wb'))
                print(f"bz2:{time()-t}s size {os.path.getsize(rel_path+'.bz2')}")
                t=time()
                fp = np.memmap(rel_path+".p", dtype='complex64', mode='w+', shape=(4,180,180))
                fp=D_matrix
                print(f"memmap:{time()-t}s size {os.path.getsize(rel_path+'.p')}")
                t=time()
                h5f = h5py.File(rel_path+'.h5', 'w')
                h5f.create_dataset('dataset_1', data=D_matrix, compression="lzf", shuffle=True) #, compression_opts=9)
                h5f.close()
                print(f"hf5 lzf:{time()-t}s size {os.path.getsize(rel_path+'.h5')}")     


                
                print(f"\nLOAD {i}")
                t=time()
                D_mat=pickle.load(lzma.open(rel_path, 'rb'))
                print(f"lzma:{time()-t}s size {os.path.getsize(rel_path)}")
                t=time()
                D_mat=pickle.load(gzip.open(rel_path+".gzip", 'rb'))
                print(f"gzip:{time()-t}s size {os.path.getsize(rel_path+'.gzip')}")
                t=time()
                D_mat=pickle.load( bz2.open(rel_path+".bz2",'rb'))
                #print(D_mat.dtype, D_mat.shape)
                print(f"bz2:{time()-t}s size {os.path.getsize(rel_path+'.bz2')}")     
                t=time()
                D_mat= np.memmap(rel_path+".p", dtype='complex64', mode='r+', shape=(4,180,180))
                print(f"memmap:{time()-t}s size {os.path.getsize(rel_path+'.p')}")     
                t=time()
                h5f = h5py.File(rel_path+'.h5','r')
                D_mat = h5f['dataset_1'][:]
                h5f.close()
                print(f"hf5:{time()-t}s size {os.path.getsize(rel_path+'.h5')}")     
                print(type(D_mat), D_mat.dtype, D_mat.shape)
                print(np.allclose(D_mat, D_matrix))

                

                if D_matrix is None:
                    raise ValueError

                #append the data
                phase_vigilant['IDs'].append(ID)
                phase_vigilant['R0s'].append(float(R0))
                phase_vigilant['Zs'].append(float(Z))
                phase_vigilant['w0s'].append(float(w0))
                phase_vigilant['rel_path'].append(rel_path)
                
                
                if i%output_info_every==0:
                    display.clear_output(wait=True)
                    elapsed=time()-beg
                    print(f"["+'#'*(int(100*i/total))+' '*(100-int(100*i/total))+f"] {100*i/total:3.4}% \n\nSimulated: {i}/{total}\nElapsed time: {elapsed//3600} h {elapsed//60-(elapsed//3600)*60} min {elapsed-(elapsed//60)*60-(elapsed//3600)*60:2.4} s")
                    if i%dump_every==0:
                        # we save the progess (in order to be able to quit and resume)
                        json.dump(phase_vigilant, open( f"{output_directory}/STRUCTURE_Grid.json", "w"))
            i+=1
print("\n\nFINISHED!!!")


DUMP 1
lzma:0.10058212280273438s size 102340
gzip:0.008575677871704102s size 155722
bz2:0.019051313400268555s size 133046
memmap:0.00029397010803222656s size 1036800
hf5 lzf:0.00263214111328125s size 188019

LOAD 1
lzma:0.006185054779052734s size 102340
gzip:0.0016970634460449219s size 155722
bz2:0.01105499267578125s size 133046
memmap:8.392333984375e-05s size 1036800
hf5:0.001087188720703125s size 188019
<class 'numpy.ndarray'> float32 (2, 180, 180)
True

DUMP 2
lzma:0.09974026679992676s size 102104
gzip:0.010133028030395508s size 153428
bz2:0.018419981002807617s size 134295
memmap:0.0002295970916748047s size 1036800
hf5 lzf:0.0027184486389160156s size 189092

LOAD 2
lzma:0.0060346126556396484s size 102104
gzip:0.0016674995422363281s size 153428
bz2:0.010317087173461914s size 134295
memmap:0.00021409988403320312s size 1036800
hf5:0.0009489059448242188s size 189092
<class 'numpy.ndarray'> float32 (2, 180, 180)
True

DUMP 3
lzma:0.10077142715454102s size 100620
gzip:0.01161551475524902

KeyboardInterrupt: 

In [ ]:
%%time
for i in range(1000):
    a=np.abs(D_mat)**2

In [ ]:
%%time
for i in range(1000):
    a=(D_mat*D_mat.conjugate()).real

In [ ]:
%%time
for i in range(1000):
    a=D_mat.real**2+D_mat.imag**2

In [ ]:
%%time
for i in range(1000):
    a=D_mat.real**2
    a+=D_mat.imag**2

In [ ]:
np.allclose( (D_mat*D_mat.conjugate()).real, D_mat.real**2+D_mat.imag**2, rtol=0)